In [1]:
# Import libraries
import json
import numpy as np
import pandas as pd

In [2]:
# Select technology
TECH = 'B'

In [3]:
# Load settings for technology
with open(f"../data/tech{TECH}/settings.json") as sfile:
    settings = json.load(sfile)

In [4]:
# Load tech B time series data
colnames = ["addr", "time", "r", "g"]
data = pd.read_csv(f"../data/tech{TECH}/relaxdata.tsv.gz", names=colnames, sep='\t')

In [5]:
# Check data
data.head()
data.tail()

,addr,time,r,g
71260894,10480,1.626754e+09,9522.314500,0.000105
71260895,10482,1.626754e+09,9253.414362,0.000108
71260896,10484,1.626754e+09,9722.581455,0.000103
71260897,10486,1.626754e+09,316811.587540,0.000003
71260898,10488,1.626754e+09,8955.346274,0.000112


In [6]:
# Subtract initial time and compute range and gi
data["time"] -= data.groupby("addr")["time"].transform("first") 
data["gi"] = data.groupby("addr")["g"].transform("first")
data["range"] = np.int32(data["gi"] / settings["gmax"] * 32)

# For tech B, subtract and make new virtual cells
if TECH == 'B':
    data.drop(data[(data["time"] < 10)].index, inplace=True) # & (data["time"] > 1)
    # data.loc[data["time"] >= 10, "addr"] += 1000000

    # Do again
    data["time"] -= data.groupby("addr")["time"].transform("first") 
    data["gi"] = data.groupby("addr")["g"].transform("first")
    data["range"] = np.int32(data["gi"] / settings["gmax"] * 32)

In [7]:
# Check data as needed
data.head()
# data.groupby('range')['range'].count() / 5500

,addr,time,r,g,gi,range
5505,0,0.0,416797.383397,0.000002,0.000002,0
5506,2,0.0,94570.348039,0.000011,0.000011,2
5507,4,0.0,64998.894320,0.000015,0.000015,3
5508,6,0.0,62715.065513,0.000016,0.000016,3
5509,8,0.0,82504.325744,0.000012,0.000012,3


In [8]:
# Get times
tdata = []
for time in settings["times"]:
    idx = (data["time"] - time).abs().groupby(data["addr"]).idxmin()
    d = data[data.index.isin(idx)]
    print(len(d))
    d = d[(d["time"] <= time*1.2) & (d["time"] >= time*0.8)]
    print(len(d))
    d["timept"] = time
    tdata.append(d)
tdata = pd.concat(tdata)

16384
16384
16384
14419
16384
16384
16384
16384
16384
16384


In [9]:
# Write to file
fopts = {"sep": '\t', "header": False, "index": False}
tdata.to_csv(f"../data/tech{TECH}/relaxdata.min.tsv.gz", **fopts)